<a href="https://colab.research.google.com/github/HYUNSOOLEE-6839/colab-Deep-Learning/blob/main/51_%EB%82%98%EB%B9%84%EC%95%BC_%EC%A0%84%EC%B2%B4%ED%9D%90%EB%A6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 순환신경망 - 나비야

##### 알파벳은 음계를 나타내며, 숫자는 음의 길이를 나타냅니다.

##### c(도), d(레), e(미), f(파), g(솔), a(라), b(시)
##### 4(4분음표), 8(8분음표)

##### g8 e8 e4 f8 d8 : 1~4번째 음표, 5번째 음표
##### e8 e4 f8 d8 d4 : 2~5번째 음표, 6번째 음표


In [1]:
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

##### 사전을 이용해서 순차적인 음표를 우리가 지정한 윈도우 크기만큼 잘라 데이터셋을 생성하는 함수를 정의

In [2]:
import numpy as np

def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

##### seq라는 변수에 “나비야” 곡 전체 음표를 저장한 다음, seq2dataset() 함수를 하여 dataset를 생성. 
##### 데이터셋은 앞서 정의한 사전에 따라 숫자로 변환되어 생성됨.

In [3]:
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

dataset = seq2dataset(seq, window_size = 4)

print(dataset.shape)

(50, 5)


### 학습과정
##### 파란색 박스가 입력값이고, 빨간색 박스가 우리가 원하는 출력값
##### 1~4번째 음표를 데이터로 5번째 음표를 라벨값으로 학습
##### 다음에는 2~5번째 음표를 데이터로 6번째 음표를 라벨값으로 학습
#####이후 한 음표씩 넘어가면서 노래 끝까지 학습

In [5]:
# http://tykimos.github.io/warehouse/2017-4-9-RNN_Layer_Talk_5.png

### 예측과정
#### 1) 한 스텝 예측
##### 한 스텝 예측이란 실제 음표 4개를 입력하여 다음 음표 1개를 예측하는 것을 반복하는 것. 
##### 이 방법에서는 모델의 입력값으로는 항상 실제 음표가 들어감.

- 모델에 t0, t1, t2, t3를 입력하면 y0 출력.
- 모델에 t1, t2, t3, t4를 입력하면 y1 출력
- 모델에 t2, t3, t4, t5를 입력하면 y2 출력
- 이 과정을 y49 출력까지 반복

In [6]:
# http://tykimos.github.io/warehouse/2017-4-9-RNN_Layer_Talk_6.png

#### 2) 곡 전체 예측
##### 곡 전체 예측이란 입력된 초가 4개 음표만을 입력으로 곡 전체를 예측. 
##### 초반부가 지나면, 예측값만으로 모델에 입력되어 다음 예측값이 나오는 방식. 
##### 그야말로 “나비야~ 나”까지 알려주면 나머지까지 모두 연주를 하는 것. 
##### 만약 중간에 틀린 부분이 생긴다면, 이후 음정, 박자는 모두 이상하게 될 가능성이 많음. 
##### 예측 오류가 누적되는 것이겠죠.

- 모델에 t0, t1, t2, t3를 입력하면 y0 출력.
- 예측값인 y0를 t4라고 가정하고, 모델에 t1, t2, t3, t4을 입력하면 y1 출력 
- 예측값인 y1을 t5라고 가정하고, 모델에 t2, t3, t4(예측값), t5(예측값)을 입력하면 y2 출력.
- 이 과정을 y49 출력까지 반복.

In [7]:
# http://tykimos.github.io/warehouse/2017-4-9-RNN_Layer_Talk_7.png